# **Tensorflow: Engineering a Kerr-based Deterministic Cubic Phase Gate via Gaussian Operations in Strawberry Fields**

[Tutorial: SF and Optimization](https://strawberryfields.ai/photonics/demos/run_tutorial_machine_learning.html)

In [41]:
import strawberryfields as sf
from strawberryfields.ops import *
import tensorflow as tf
import numpy as np

Flow:
1) set $\alpha$
2) calculate parameters based on $\alpha$
3) run program
4) get error
5) backpropagate

In [ ]:
# defining constant parameters
N = tf.constant(5)
cutoff_dim = 30
backend = "tf"


""" What we change """
# Cubic Phase Gate
gamma = tf.constant(0.7, dtype=tf.float32)  # target gate angle

# Squeezing
lamdB = tf.constant(14.0, dtype=tf.float32)  # squeezing level in dB

# Nonlinearity
chi = tf.constant(0.1, dtype=tf.float32)  # nonlinearity strength

""" The variable """
# Displacement
tf_alpha = tf.Variable(4, dtype=tf.float32)  # displacement amplitude

""" Define operations """
def get_params(alpha):
    lam = tf.sqrt(tf.pow(10.0, lamdB / 10.0))     # squeezing parameter
    rsq = -tf.math.log(lam)                       # squeezing parameter in r form
    sqphi = tf.constant(0.0, dtype=tf.float32)

    # Displacement
    rd = tf.abs(alpha)                              # displacement amplitude in r form
    phi = tf.constant(0.0, dtype=tf.float32)

    # Parameters evolving from alpha
    detuning = 3.0 * chi * tf.pow(alpha, 2) - chi  # detuning for the cubic phase gate
    drive = -2.0 * chi * tf.pow(alpha, 3)          # drive for the cubic phase gate

    # Gate time calculations
    t = tf.sqrt(2.0) * gamma / (chi * alpha * tf.pow(lam, 3))
    dt = t / tf.cast(N, tf.float32)

    # Parameters for the cubic phase gate step
    nkappa = (chi / 2.0) * dt    # for Kerr gate
    nkerrr = (-chi / 2.0) * dt   # rotation for Kerr operator
    nrphi = -detuning * dt       # for second rotation
    nr = drive * dt              # magnitude of displacement in nonlinear step
    ndphi = tf.constant(3 * np.pi / 2, dtype=tf.float32)  # direction angle for displacement
    
    return rsq, sqphi, rd, phi, chi, nkappa, nkerrr, nrphi, nr, ndphi

In [46]:
""" Find ground truth """
# Build the target program (ideal cubic phase gate)
prog_target = sf.Program(1)
with prog_target.context as q:
    Vgate(gamma) | q[0]

eng_target = sf.Engine(backend="tf", backend_options={"cutoff_dim": cutoff_dim})
target_state = eng_target.run(prog_target).state

""" Define loss """
def loss_fn(simstate):
    return tf.linalg.norm(simstate.ket() - target_state.ket()) # returns L2 norm of difference

In [47]:
# initialize engine and program objects
eng = sf.Engine(backend="tf", backend_options={"cutoff_dim": cutoff_dim})
prog = sf.Program(1)

# define parametrization of circuit
rsq, sqphi, rd, phi, chi, nkappa, nkerrr, nrphi, nr, ndphi = prog.params("rsq", "sqphi", "rd", "phi", "chi", "nkappa", "nkerrr", "nrphi", "nr", "ndphi")

# define program
with prog.context as q:
        # Direct operations: squeezing then displacement
        Sgate(rsq, sqphi) | q[0]
        Dgate(rd, phi) | q[0]
        
        # Nonlinear step(s)
        for i in range(N):
            Rgate(nkerrr) | q[0]
            Kgate(nkappa) | q[0]
            Rgate(nrphi) | q[0]
            Dgate(nr, ndphi) | q[0]
        
        # Inverse operations: displacement then squeezing
        Dgate(-rd, phi) | q[0]
        Sgate(-rsq, sqphi) | q[0]

opt = tf.keras.optimizers.Adam(learning_rate=0.01)
steps = 50

for step in range(steps):

    # reset the engine if it has already been executed
    if eng.run_progs:
        eng.reset()

    with tf.GradientTape() as tape:
        # calculate parameters
        rsq, sqphi, rd, phi, chi, nkappa, nkerrr, nrphi, nr, ndphi = get_params(tf_alpha)

        # execute the engine
        simstate = eng.run(prog, args={"rsq": rsq, "sqphi": sqphi, "rd": rd, "phi": phi, "chi": chi, "nkappa": nkappa, "nkerrr": nkerrr, "nrphi": nrphi, "nr": nr, "ndphi": ndphi}).state
        # get the probability of fock state |1>
        loss = loss_fn(simstate)

    gradients = tape.gradient(loss, [tf_alpha])
    opt.apply_gradients(zip(gradients, [tf_alpha]))
    print("Error at step {}: {}".format(step, loss.numpy()))



ValueError: Attempt to convert a value (-0.000277685470438163) with an unsupported type (<class 'sympy.core.numbers.Float'>) to a Tensor.